In [1]:
import pandas as pd
import numpy as np
import urllib
import time # понадобится, чтобы вставлять паузы в выполнение кода

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## 1. Semi-manual version of process

In [87]:
driver = webdriver.Chrome('chromedriver.exe') # Откроем браузер через драйвер
driver.get('https://2gis.ru/astrakhan/') # В открывшемся браузере перейдем на сайт 2ГИС

In [110]:
categories = pd.read_excel('categories.xlsx')
categories.head()

,category
0,Бани
1,Кинотеатры
2,Ночные клубы
3,Бары
4,Кафе


In [88]:
# Определим поле для отправки поисковых запросов
search_bar = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[2]/form/div/input')

In [89]:
# Выведем список заведений
#search_bar.send_keys('Поесть')
search_bar.send_keys(categories.category[2])
search_bar.send_keys(Keys.ENTER)

In [90]:
# Для переключения между страницами нужно понять, сколько их всего
num_of_pages = int(np.ceil(int(driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[1]/header/div[3]/div/div[1]/div/div/a/span').text) / 12))
print('Всего страниц организаций в поисковой выдаче: %s' % num_of_pages)

Всего страниц организаций в поисковой выдаче: 65


Переключение между организациями будет происходить через подставление порядкового номера в XPATH

Например:
* XPATH первой организации: //*[@id="root"]/div/.../div/div[1]/div/div[2]
* XPATH второй организации: //*[@id="root"]/div/.../div/div[2]/div/div[2]

In [60]:
# Инициализируем таблицы
df_result = pd.DataFrame()
df_errors = pd.DataFrame()

In [61]:
for i in range(1, num_of_pages+1):
    try:
        #company_cards = driver.find_elements_by_xpath("//a[contains(@href, '/astrakhan/firm/')]")
        company_cards = driver.find_elements_by_class_name('_1h3cgic')
        
        for j in range(len(company_cards)):
            try:
                company_cards[j].click() # 1. Откроем карточку организации
                time.sleep(0.5)

                df_temp = pd.DataFrame({
                    'name': driver.find_element_by_tag_name('h1').text, # 2. Сохраним название организации
                    'address': driver.find_element_by_xpath("//a[contains(@href, '/astrakhan/geo/')]").text # 3. Сохраним адрес
                }, index=[0])
                
                close_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[1]/div[2]/div/div[1]/div'
                driver.find_element_by_xpath(close_xp).click() # Закроем карточку

                df_result = df_result.append(df_temp)
                time.sleep(0.5)

            except Exception:
                df_temp = pd.DataFrame({'i': i, 'j': j}, index=[0])
                df_errors = df_errors.append(df_temp)

        #next_page_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div[3]/div[2]/div[2]'
        driver.find_element_by_class_name('_n5hmn94').click() # Переключимся на следующую страницу
        print(i)
        time.sleep(0.7) # Подождем, чтоб список прогрузился
    
    except Exception:
        pass

In [62]:
print('Кол-во успешно записанных организаций: %s' % df_result.shape[0])
print('Кол-во ошибок: %s' % df_errors.shape[0])

Кол-во успешно записанных организаций: 23
Кол-во ошибок: 1


In [71]:
df_result.head()

,name,address
0,Чикаго,"Воробьёва проезд, 12 к2"
0,Gentleman club Vogue,"Савушкина, 4 к6"
0,Чикаго,"Ахматовская, 14"
0,JET SET,"Эспланадная, 3"
0,Монако,"Воробьёва проезд, 7а"


In [41]:
df_errors.head()

,i,j
0,1,3
0,1,4
0,1,5
0,1,6
0,1,7


In [79]:
df_result.to_csv('addresses.csv', index=False)

## 2. Automated object-oriented version

In [131]:
class session_of_2gis:
    def __init__(self, company_cards_class_name: str = '_1h3cgic'):
        self.search_bar_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[2]/form/div/input'
        self.number_of_pages_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[1]/header/div[3]/div/div[1]/div/div/a/span'
        self.company_cards_class_name = company_cards_class_name
        self.switch_page_button_xp = "//*[name()='svg'][contains(@style, '-90deg')]"
        
    def scrape_organizations(self, category: str):
        driver = webdriver.Chrome('chromedriver.exe') # Откроем браузер через драйвер
        driver.get('https://2gis.ru/astrakhan/') # В открывшемся браузере перейдем на сайт 2ГИС

        # Определим поле для отправки поисковых запросов
        search_bar = driver.find_element_by_xpath(self.search_bar_xp)

        # Выведем список заведений
        search_bar.send_keys(category)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(1)

        num_of_pages = int(np.ceil(int(driver.find_element_by_xpath(self.number_of_pages_xp).text) / 12))
        
        # Соглашаемся с новой политикой конфидециальности (в дальнейшем убрать/обернуть в try/except?)
        driver.find_element_by_xpath('//*[@id="root"]/div/div/div[3]/footer/div[1]/div[2]/button').click()

        df_result = pd.DataFrame()

        for i in range(1, num_of_pages+1):
            try:
                company_cards = driver.find_elements_by_class_name(self.company_cards_class_name)

                for j in range(len(company_cards)):
                    try:
                        company_cards[j].click() # 1. Откроем карточку организации
                        time.sleep(0.5)

                        df_temp = pd.DataFrame({
                            'name': driver.find_element_by_tag_name('h1').text, # 2. Сохраним название организации
                            'address': driver.find_element_by_xpath("//a[contains(@href, '/astrakhan/geo/')]").text # 3. Сохраним адрес
                        }, index=[0])

                        close_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[1]/div[2]/div/div[1]/div'
                        driver.find_element_by_xpath(close_xp).click() # Закроем карточку

                        df_result = df_result.append(df_temp)
                        time.sleep(0.5)

                    except Exception:
                        pass

                driver.find_element_by_xpath(self.switch_page_button_xp).click() # Переключимся на следующую страницу
                time.sleep(0.7) # Подождем, чтоб список прогрузился

            except Exception:
                pass

        # Нужно внедрить reg exp для исправления названий категорий
        df_result.to_csv('data/addresses_' + category.replace(' ', '_').replace('-', '_').replace('/', '').replace(',', '_') + '.csv', index=False)
        driver.close()

In [132]:
session = session_of_2gis()

In [133]:
for cat in categories.category[5:]:
    session.scrape_organizations(category=cat)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[1]/header/div[3]/div/div[1]/div/div/a/span"}
  (Session info: chrome=85.0.4183.121)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.18362 x86_64)
